In [236]:
import os
import glob
from datetime import datetime
import re
import shutil
import random
import matplotlib.pyplot as plt
import numpy as np

In [237]:
imgs = glob.glob(r"C:\Users\corpa\Downloads\pyronear_ds_03_2024\pyronear_ds_03_2024\labels/**/*.txt")
imgs.sort()
print(len(imgs))
# quitar los que contengan hpwren
imgs = [img for img in imgs if "hpwren" not in img]
print(len(imgs))

15998
10145


In [238]:
imgs = glob.glob(r"C:\Users\corpa\Downloads\pyronear_ds_03_2024\pyronear_ds_03_2024\labels/**/*.txt")
imgs = [img for img in imgs if "hpwren" not in img]
imgs.sort()
print(len(imgs))

fires = {}
fire_idx = -1
t0 = datetime.now()
for file in imgs:
    match = re.search(r"(\d{4}_\d{2}_\d{2}T\d{2}_\d{2}_\d{2})", file)
    if match is not None:
        t = datetime.strptime(match.group(), "%Y_%m_%dT%H_%M_%S")

        if abs((t-t0).total_seconds()) > 10*59:
            fire_idx += 1

        t0 = t

        if fire_idx in fires.keys():
            fires[fire_idx].append(file)
        else:
            fires[fire_idx] = [file]

len(fires)  

10145


366

In [239]:
to_del = []
for k, v in fires.items():
    if len(v) <10:
        to_del.append(k)
    else:
        keep = []
        for file in v:
            with open(file, 'r') as f:
                lines = f.readlines()

            if len(lines):
                keep.append(file)

        if len(keep) <10:
            to_del.append(k)

In [240]:
for k in to_del:
    del fires[k]

In [243]:
imagenes =  0
for k, v in fires.items():
    imagenes += len(v)
print(imagenes)
len(fires)

8359


189

In [244]:
import random

# Definir proporciones
train_split = 0.8
val_split = 0.1
test_split = 0.1

# Inicializar diccionarios para cada conjunto
train = {}
val = {}
test = {}

# Número total de incendios
n_fires = len(fires)

# Crear una lista de claves de 'fires' y barajarla
shuffled_keys = list(fires.keys())
random.shuffle(shuffled_keys)

# Calcular los índices de corte para cada conjunto
train_limit = int(n_fires * train_split)
val_limit = train_limit + int(n_fires * val_split)

# Repartir los incendios en los conjuntos
for i, k in enumerate(shuffled_keys):
    if i < train_limit:
        train[k] = fires[k]  # Asignar al conjunto de entrenamiento
    elif i < val_limit:
        val[k] = fires[k]  # Asignar al conjunto de validación
    else:
        test[k] = fires[k]  # Asignar al conjunto de prueba

# Verificar el tamaño de cada conjunto
print(f"Train set size: {len(train)}")
print(f"Validation set size: {len(val)}")
print(f"Test set size: {len(test)}")



Train set size: 151
Validation set size: 18
Test set size: 20


In [245]:
# Inicializar diccionarios para almacenar los subconjuntos de cada partición
train_sets = {}
val_sets = {}
test_sets = {}

random.seed(33)

# Función para crear subconjuntos
def create_subsets(fires_dict, sets, start_idx=0):
    for k, v in fires_dict.items():
        n = len(v)

        if n >= 10:
            # 1. Tomar la primera secuencia de 4 archivos
            sets[start_idx] = v[0:10]
            start_idx += 1

            # 2. Tomar una secuencia de 4 archivos a la mitad de la lista
            mid_idx = (n // 2) - 2  # Calcula el índice de la mitad ajustado para una ventana de 4 archivos
            mid_idx = max(0, min(mid_idx, n - 10))  # Asegúrate de que el índice esté dentro de los límites
            sets[start_idx] = v[mid_idx:mid_idx+10]
            start_idx += 1

            # 3. Tomar una secuencia de 4 archivos al final
            sets[start_idx] = v[-10:]
            start_idx += 1


    return sets, start_idx

# Crear subconjuntos para train, val, y test
train_sets, _ = create_subsets(train, train_sets)
val_sets, _ = create_subsets(val, val_sets)
test_sets, _ = create_subsets(test, test_sets)

# Verificar el tamaño total de los subconjuntos creados
print(f"Train subsets: {len(train_sets)}")
print(f"Validation subsets: {len(val_sets)}")
print(f"Test subsets: {len(test_sets)}")

Train subsets: 453
Validation subsets: 54
Test subsets: 60


In [248]:
import os
import shutil

# Función para copiar archivos de etiquetas e imágenes
def copy_files(sets, set_name):
    for k, v in sets.items():
        for label_file in v:
            # Crear nueva ruta para el archivo de etiqueta en la carpeta temporal
            new_label_file = f"C:/nico/wildfire2024/data/temporal_ds/labels/{set_name}/{1}/{k}/{os.path.basename(label_file)}"
            os.makedirs(os.path.dirname(new_label_file), exist_ok=True)
            shutil.copy(label_file, new_label_file)

            # Generar la ruta para el archivo de imagen correspondiente
            img_file = label_file.replace("labels", "images").replace(".txt", ".jpg")
            new_img_file = f"C:/nico/wildfire2024/data/temporal_ds/images/{set_name}/{1}/{k}/{os.path.basename(img_file)}"
            os.makedirs(os.path.dirname(new_img_file), exist_ok=True)
            shutil.copy(img_file, new_img_file)

# Copiar archivos para train, val y test
copy_files(train_sets, "train")
copy_files(val_sets, "val")
copy_files(test_sets, "test")

print("Archivos copiados correctamente.")

Archivos copiados correctamente.


In [250]:
imgs = glob.glob(r"C:\Users\corpa\Downloads\DS_fp\DS_fp/labels/*.txt")
imgs.sort()
print(len(imgs))

fires = {}
fire_idx = -1
t0 = datetime.now()
for file in imgs:
    match = re.search(r"(\d{4}_\d{2}_\d{2}T\d{2}_\d{2}_\d{2})", file)
    if match is not None:
        t = datetime.strptime(match.group(), "%Y_%m_%dT%H_%M_%S")

        if abs((t-t0).total_seconds()) > 2*59:
            fire_idx += 1

        t0 = t

        if fire_idx in fires.keys():
            fires[fire_idx].append(file)
        else:
            fires[fire_idx] = [file]

len(fires)

50795


4497

In [251]:
to_del = []
for k, v in fires.items():
    if len(v) <10:
        to_del.append(k)
    else:
        keep = []
        for file in v:
            with open(file, 'r') as f:
                lines = f.readlines()

            if len(lines):
                keep.append(file)

        if len(keep) <10:
            to_del.append(k)

In [252]:
for k in to_del:
    del fires[k]

len(fires)

1376

In [253]:
fire_train_size = len(glob.glob(r"C:\nico\wildfire2024\data\temporal_ds/labels/train/1/*"))
fire_val_size = len(glob.glob(r"C:\nico\wildfire2024\data\temporal_ds/labels/val/1/*"))
fire_test_size = len(glob.glob(r"C:\nico\wildfire2024\data\temporal_ds/labels/test/1/*"))
fire_train_size, fire_val_size, fire_test_size

(453, 54, 60)

In [255]:
random.seed(33)
indexes = list(fires.keys())
random.shuffle(indexes)
sets_tain = indexes[:fire_train_size]
sets_val = indexes[fire_train_size:fire_train_size+fire_val_size]
sets_test = indexes[-fire_test_size:]
len(sets_tain), len(sets_val), len(sets_test)

(453, 54, 60)

In [258]:
for k in sets_tain:
    v = fires[k]
    idx = random.randint(0,len(v)-10)
    v = v[idx:idx+10]
    for label_file in v:
   
        new_label_file = f"C:/nico/wildfire2024/data/temporal_ds/labels/train/0/{k}/{os.path.basename(label_file)}"
        os.makedirs(os.path.dirname(new_label_file), exist_ok=True)
        shutil.copy(label_file, new_label_file)

        img_file = label_file.replace("labels","images").replace(".txt",".jpg")
        
        new_img_file = f"C:/nico/wildfire2024/data/temporal_ds/images/train/0/{k}/{os.path.basename(img_file)}"
        os.makedirs(os.path.dirname(new_img_file), exist_ok=True)
        shutil.copy(img_file, new_img_file)


for k in sets_val:
    v = fires[k]
    idx = random.randint(0,len(v)-10)
    v = v[idx:idx+10]
    for label_file in v:
   
        new_label_file = f"C:/nico/wildfire2024/data/temporal_ds/labels/val/0/{k}/{os.path.basename(label_file)}"
        os.makedirs(os.path.dirname(new_label_file), exist_ok=True)
        shutil.copy(label_file, new_label_file)

        img_file = label_file.replace("labels","images").replace(".txt",".jpg")
        
        new_img_file = f"C:/nico/wildfire2024/data/temporal_ds/images/val/0/{k}/{os.path.basename(img_file)}"
        os.makedirs(os.path.dirname(new_img_file), exist_ok=True)
        shutil.copy(img_file, new_img_file)

for k in sets_test:
    v = fires[k]
    idx = random.randint(0,len(v)-10)
    v = v[idx:idx+10]
    for label_file in v:
   
        new_label_file = f"C:/nico/wildfire2024/data/temporal_ds/labels/test/0/{k}/{os.path.basename(label_file)}"
        os.makedirs(os.path.dirname(new_label_file), exist_ok=True)
        shutil.copy(label_file, new_label_file)

        img_file = label_file.replace("labels","images").replace(".txt",".jpg")
        
        new_img_file = f"C:/nico/wildfire2024/data/temporal_ds/images/test/0/{k}/{os.path.basename(img_file)}"
        os.makedirs(os.path.dirname(new_img_file), exist_ok=True)
        shutil.copy(img_file, new_img_file)